In [1]:
# ============================================================
# HDS Chess Explainer (Kaggle 1-cell / reproducibility-first)
# - Objective → Axes → Alternatives → Risk(EVR) → Choice
# - Engine-agnostic: engine候補手ログを差し込める／無ければ全合法手から自動生成
# - Deterministic: 安定ソート + UCIタイブレーク（乱数なし）
# ============================================================

# --- bootstrap: install python-chess if missing ---
try:
    import chess
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-chess", "-q"])
    import chess

from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Any


# -----------------------------
# 型（Data Types）
# -----------------------------

ColorStr = str  # "white" | "black"


@dataclass(frozen=True)
class CandidateMove:
    """外部エンジンから受け取る候補手の最小表現（任意項目は空でOK）"""
    move_uci: str
    engine_eval_cp: Optional[int] = None  # エンジン評価（cp想定、白視点が+）
    policy: Optional[float] = None        # 方策確率など（任意）
    pv_uci: Optional[str] = None          # principal variation（任意）
    label: Optional[str] = None           # "best"/"second" など（任意）


@dataclass(frozen=True)
class FeatureVector:
    """解釈可能な特徴量（perspective視点で + が良い）"""
    material: float
    king_safety: float
    activity: float
    center_control: float
    pawn_structure: float
    piece_safety: float     # 自駒の脆弱度（低いほど良い）を + にした指標
    threats: float          # 短期の脅威（相手駒への圧力/チェック）を + にした指標

    def as_dict(self) -> Dict[str, float]:
        return {
            "material": self.material,
            "king_safety": self.king_safety,
            "activity": self.activity,
            "center_control": self.center_control,
            "pawn_structure": self.pawn_structure,
            "piece_safety": self.piece_safety,
            "threats": self.threats,
        }


@dataclass(frozen=True)
class RiskEVR:
    """EVR（Expected Value Range）っぽい“レンジ”の簡易表現"""
    base: float     # after_score（中心）
    downside: float # 下振れリスク（大きいほど悪い）
    upside: float   # 上振れ余地（大きいほど良い）

    @property
    def low(self) -> float:
        return self.base - self.downside

    @property
    def high(self) -> float:
        return self.base + self.upside


@dataclass(frozen=True)
class CandidateAnalysis:
    """候補手の分析結果（HDS Alternatives 用）"""
    move_uci: str
    move_san: str
    engine_eval_cp: Optional[int]
    score_before: float
    score_after: float
    score_delta: float
    features_before: FeatureVector
    features_after: FeatureVector
    deltas: Dict[str, float]
    evr: RiskEVR
    gap_to_best: float
    eval_ratio: float
    risk_category: str
    comment_ja: str


@dataclass(frozen=True)
class HDSReport:
    """最終出力（ログ＋表＋要約）"""
    best_move_uci: str
    best_move_san: str
    perspective: ColorStr
    table: List[CandidateAnalysis]
    hds_log: str
    executive_summary_ja: str


# -----------------------------
# 評価モデル（軽量・解釈可能）
# -----------------------------

PIECE_VALUES = {
    chess.PAWN: 1.0,
    chess.KNIGHT: 3.0,
    chess.BISHOP: 3.0,
    chess.ROOK: 5.0,
    chess.QUEEN: 9.0,
    chess.KING: 0.0,
}

CENTER_SQUARES = [chess.D4, chess.E4, chess.D5, chess.E5]


@dataclass(frozen=True)
class ScoreWeights:
    """特徴量の重み（HDS Axes の中核）"""
    material: float = 1.00
    king_safety: float = 0.85
    activity: float = 0.45
    center_control: float = 0.25
    pawn_structure: float = 0.25
    piece_safety: float = 0.55
    threats: float = 0.35

    def as_dict(self) -> Dict[str, float]:
        return {
            "material": self.material,
            "king_safety": self.king_safety,
            "activity": self.activity,
            "center_control": self.center_control,
            "pawn_structure": self.pawn_structure,
            "piece_safety": self.piece_safety,
            "threats": self.threats,
        }


class SimpleFeatureEvaluator:
    """
    軽量で決定的な“特徴量抽出器”
    - エンジン精度ではなく「説明可能性」優先
    - perspective 視点で + が良い
    """

    def evaluate(self, board: chess.Board, perspective: bool) -> FeatureVector:
        return FeatureVector(
            material=self._material(board, perspective),
            king_safety=self._king_safety(board, perspective),
            activity=self._activity(board, perspective),
            center_control=self._center_control(board, perspective),
            pawn_structure=self._pawn_structure(board, perspective),
            piece_safety=self._piece_safety(board, perspective),
            threats=self._threats(board, perspective),
        )

    def _material(self, board: chess.Board, perspective: bool) -> float:
        s = 0.0
        for _, p in board.piece_map().items():
            v = PIECE_VALUES.get(p.piece_type, 0.0)
            s += v if p.color == perspective else -v
        return s

    def _king_safety(self, board: chess.Board, perspective: bool) -> float:
        ksq = board.king(perspective)
        if ksq is None:
            return -10.0

        ring = chess.SquareSet(chess.BB_KING_ATTACKS[ksq] | chess.BB_SQUARES[ksq])

        shield = 0
        for sq in ring:
            p = board.piece_at(sq)
            if p and p.piece_type == chess.PAWN and p.color == perspective:
                shield += 1

        pressure = 0.0
        for sq in ring:
            pressure += len(board.attackers(not perspective, sq))

        # ざっくり open-file penalty near king
        file_idx = chess.square_file(ksq)
        files = [file_idx]
        if file_idx - 1 >= 0:
            files.append(file_idx - 1)
        if file_idx + 1 <= 7:
            files.append(file_idx + 1)

        open_pen = 0.0
        fp = board.pieces(chess.PAWN, perspective)
        for f in files:
            has_pawn = any(chess.square_file(sq) == f for sq in fp)
            if not has_pawn:
                open_pen += 0.4

        return (0.9 * shield) - (0.25 * pressure) - open_pen

    def _activity(self, board: chess.Board, perspective: bool) -> float:
        tmp = board.copy(stack=False)
        tmp.turn = perspective
        legal = tmp.legal_moves.count()

        attacked = set()
        for sq, p in tmp.piece_map().items():
            if p.color != perspective:
                continue
            for t in tmp.attacks(sq):
                attacked.add(t)

        return 0.05 * float(legal) + 0.015 * float(len(attacked))

    def _center_control(self, board: chess.Board, perspective: bool) -> float:
        my = sum(len(board.attackers(perspective, sq)) for sq in CENTER_SQUARES)
        op = sum(len(board.attackers(not perspective, sq)) for sq in CENTER_SQUARES)
        return float(my - op)

    def _pawn_structure(self, board: chess.Board, perspective: bool) -> float:
        pawns = board.pieces(chess.PAWN, perspective)
        if not pawns:
            return 0.0

        files: Dict[int, List[int]] = {}
        for sq in pawns:
            f = chess.square_file(sq)
            files.setdefault(f, []).append(sq)

        doubled = 0.0
        isolated = 0.0
        for f, sqs in files.items():
            if len(sqs) > 1:
                doubled += 0.30 * (len(sqs) - 1)
            has_left = (f - 1) in files
            has_right = (f + 1) in files
            if not has_left and not has_right:
                isolated += 0.30

        passed = 0.0
        for sq in pawns:
            if self._is_passed_pawn(board, sq, perspective):
                passed += 0.35

        return passed - doubled - isolated

    def _is_passed_pawn(self, board: chess.Board, sq: chess.Square, perspective: bool) -> bool:
        file_idx = chess.square_file(sq)
        rank_idx = chess.square_rank(sq)
        direction = 1 if perspective == chess.WHITE else -1
        enemy = not perspective

        for f in range(file_idx - 1, file_idx + 2):
            if f < 0 or f > 7:
                continue
            r = rank_idx + direction
            while 0 <= r <= 7:
                tsq = chess.square(f, r)
                p = board.piece_at(tsq)
                if p and p.piece_type == chess.PAWN and p.color == enemy:
                    return False
                r += direction
        return True

    def _piece_safety(self, board: chess.Board, perspective: bool) -> float:
        penalty = 0.0
        for sq, p in board.piece_map().items():
            if p.color != perspective:
                continue
            if p.piece_type == chess.KING:
                continue

            attackers = len(board.attackers(not perspective, sq))
            if attackers == 0:
                continue

            defenders = len(board.attackers(perspective, sq))
            excess = max(0, attackers - defenders)
            v = PIECE_VALUES.get(p.piece_type, 0.0)
            penalty += excess * (0.15 * v)

        return -penalty  # higher is better

    def _threats(self, board: chess.Board, perspective: bool) -> float:
        bonus = 0.0
        # 相手駒への圧力（攻撃＞防御ならボーナス）
        for sq, p in board.piece_map().items():
            if p.color == perspective:
                continue
            if p.piece_type == chess.KING:
                continue

            my_atk = len(board.attackers(perspective, sq))
            if my_atk == 0:
                continue
            opp_def = len(board.attackers(not perspective, sq))
            v = PIECE_VALUES.get(p.piece_type, 0.0)
            advantage = max(0, my_atk - opp_def)
            bonus += advantage * (0.10 * v)

        # チェック可能性（1つでもチェックがあれば加点）
        tmp = board.copy(stack=False)
        tmp.turn = perspective
        moves = sorted([m.uci() for m in tmp.legal_moves])  # deterministic
        for u in moves:
            m = chess.Move.from_uci(u)
            if tmp.gives_check(m):
                bonus += 0.25
                break

        return bonus


class LinearScorer:
    """特徴量×重みの線形スコア（説明優先・決定的）"""

    def __init__(self, weights: Optional[ScoreWeights] = None):
        self.weights = weights or ScoreWeights()

    def score(self, fv: FeatureVector) -> float:
        w = self.weights.as_dict()
        d = fv.as_dict()
        s = 0.0
        for k, wk in w.items():
            s += wk * float(d[k])
        return s


class SimpleRiskModel:
    """
    リスク（EVR）を“それっぽく”出す決定的な簡易モデル。
    - downside: king_safety↓ / piece_safety↓ / score_afterが悪い
    - upside: threats↑ / activity↑ / center↑
    """

    def __init__(self, weights: Optional[ScoreWeights] = None):
        self.weights = weights or ScoreWeights()

    def estimate(self, before: FeatureVector, after: FeatureVector, score_after: float) -> RiskEVR:
        db = before.as_dict()
        da = after.as_dict()
        delta = {k: da[k] - db[k] for k in da.keys()}

        down = 0.0
        if delta["king_safety"] < 0:
            down += (-delta["king_safety"]) * (self.weights.king_safety * 0.9)
        if delta["piece_safety"] < 0:
            down += (-delta["piece_safety"]) * (self.weights.piece_safety * 0.9)
        down += max(0.0, -score_after) * 0.05

        up = 0.0
        if delta["threats"] > 0:
            up += delta["threats"] * (self.weights.threats * 1.0)
        if delta["activity"] > 0:
            up += delta["activity"] * (self.weights.activity * 0.6)
        if delta["center_control"] > 0:
            up += delta["center_control"] * (self.weights.center_control * 0.6)

        down = float(round(down, 4))
        up = float(round(up, 4))
        return RiskEVR(base=float(round(score_after, 4)), downside=down, upside=up)


# -----------------------------
# HDS 生成（Chess）
# -----------------------------

class HDSChessExplainer:
    """
    HDS：Objective → Axes → Alternatives → Risk(EVR) → Choice
    - candidates を渡せば“エンジンログ接続”
    - candidates 無しなら全合法手を軽量評価して候補を自動生成
    """

    def __init__(
        self,
        evaluator: Optional[SimpleFeatureEvaluator] = None,
        scorer: Optional[LinearScorer] = None,
        risk_model: Optional[SimpleRiskModel] = None,
        weights: Optional[ScoreWeights] = None,
    ):
        self.evaluator = evaluator or SimpleFeatureEvaluator()
        self.scorer = scorer or LinearScorer(weights=weights)
        self.risk_model = risk_model or SimpleRiskModel(weights=weights)
        self.weights = self.scorer.weights

    def explain_position(
        self,
        fen: str,
        perspective: ColorStr = "white",
        candidates: Optional[List[CandidateMove]] = None,
        max_alternatives: int = 5,
        include_worst: int = 0,
    ) -> HDSReport:
        board0 = chess.Board(fen)
        pov = chess.WHITE if perspective.lower() == "white" else chess.BLACK

        before_fv = self.evaluator.evaluate(board0, pov)
        score_before = self.scorer.score(before_fv)

        cand_moves = self._prepare_candidates(board0, pov, candidates, max_alternatives, include_worst)

        analyses: List[CandidateAnalysis] = []
        for cm in cand_moves:
            mv = chess.Move.from_uci(cm.move_uci)
            if mv not in board0.legal_moves:
                continue

            b1 = board0.copy(stack=False)
            move_san = b1.san(mv)
            b1.push(mv)

            after_fv = self.evaluator.evaluate(b1, pov)
            score_after = self.scorer.score(after_fv)
            score_delta = score_after - score_before

            deltas = self._deltas(before_fv, after_fv)
            evr = self.risk_model.estimate(before_fv, after_fv, score_after)

            analyses.append(
                CandidateAnalysis(
                    move_uci=cm.move_uci,
                    move_san=move_san,
                    engine_eval_cp=cm.engine_eval_cp,
                    score_before=float(round(score_before, 4)),
                    score_after=float(round(score_after, 4)),
                    score_delta=float(round(score_delta, 4)),
                    features_before=before_fv,
                    features_after=after_fv,
                    deltas={k: float(round(v, 4)) for k, v in deltas.items()},
                    evr=evr,
                    gap_to_best=0.0,
                    eval_ratio=0.0,
                    risk_category="",
                    comment_ja="",
                )
            )

        if not analyses:
            raise ValueError("候補手が生成/適用できません（FENか手番を確認）。")

        use_engine = any(a.engine_eval_cp is not None for a in analyses)
        best_val = self._best_value(analyses, use_engine=use_engine, pov=pov)

        filled: List[CandidateAnalysis] = []
        for a in analyses:
            val = self._value_for_ranking(a, use_engine=use_engine, pov=pov)
            gap = best_val - val
            denom = best_val if abs(best_val) > 1e-9 else 1.0
            ratio = val / denom

            gap_cp = gap if use_engine else gap * 100.0  # score→cp風
            risk_cat = self._risk_category(gap_cp)
            comment = self._comment_ja(a, risk_cat)

            # ★FIX: a.__dict__ に gap_to_best 等が既にあるので、上書き対象キーは除外して二重指定を防ぐ
            base = dict(a.__dict__)
            for k in ("gap_to_best", "eval_ratio", "risk_category", "comment_ja"):
                base.pop(k, None)

            filled.append(
                CandidateAnalysis(
                    **base,
                    gap_to_best=float(round(gap_cp, 2)),
                    eval_ratio=float(round(ratio, 4)),
                    risk_category=risk_cat,
                    comment_ja=comment,
                )
            )

        filled_sorted = sorted(
            filled,
            key=lambda x: (-self._value_for_ranking(x, use_engine=use_engine, pov=pov), x.move_uci),
        )

        best = filled_sorted[0]
        hds_log = self._build_hds_log(
            board0=board0,
            perspective=perspective,
            use_engine=use_engine,
            score_before=score_before,
            weights=self.weights,
            analyses=filled_sorted,
            max_alternatives=max_alternatives,
        )
        summary = self._build_executive_summary_ja(
            perspective=perspective,
            use_engine=use_engine,
            analyses=filled_sorted[: max(3, min(len(filled_sorted), max_alternatives))],
        )

        return HDSReport(
            best_move_uci=best.move_uci,
            best_move_san=best.move_san,
            perspective=perspective,
            table=filled_sorted,
            hds_log=hds_log,
            executive_summary_ja=summary,
        )

    # --- internals ---

    def _prepare_candidates(
        self,
        board: chess.Board,
        pov: bool,
        candidates: Optional[List[CandidateMove]],
        max_alternatives: int,
        include_worst: int,
    ) -> List[CandidateMove]:
        if candidates:
            seen = set()
            out: List[CandidateMove] = []
            for c in candidates:
                if c.move_uci in seen:
                    continue
                seen.add(c.move_uci)
                out.append(c)
            return out

        before_fv = self.evaluator.evaluate(board, pov)
        score_before = self.scorer.score(before_fv)

        # deterministic: iterate legal moves sorted by UCI
        legal_uci = sorted([m.uci() for m in board.legal_moves])

        scored: List[Tuple[float, str]] = []
        for u in legal_uci:
            mv = chess.Move.from_uci(u)
            b1 = board.copy(stack=False)
            b1.push(mv)
            fv = self.evaluator.evaluate(b1, pov)
            s = self.scorer.score(fv)
            s += 0.15 * (s - score_before)  # 小さな“勢い”補正（決定的）
            scored.append((float(round(s, 6)), u))

        scored_sorted = sorted(scored, key=lambda t: (-t[0], t[1]))

        take_top = max(1, max_alternatives)
        top = scored_sorted[:take_top]
        worst = scored_sorted[-include_worst:] if include_worst > 0 and len(scored_sorted) > take_top else []

        picked = top + worst
        seen2 = set()
        out2: List[CandidateMove] = []
        for _, uci in picked:
            if uci in seen2:
                continue
            seen2.add(uci)
            out2.append(CandidateMove(move_uci=uci, engine_eval_cp=None))
        return out2

    def _deltas(self, before: FeatureVector, after: FeatureVector) -> Dict[str, float]:
        b = before.as_dict()
        a = after.as_dict()
        return {k: (a[k] - b[k]) for k in a.keys()}

    def _risk_category(self, gap_cp: float) -> str:
        if gap_cp <= 30:
            return "low_risk_stable"
        if gap_cp <= 120:
            return "balanced"
        return "high_risk_gambling"

    def _value_for_ranking(self, a: CandidateAnalysis, use_engine: bool, pov: bool) -> float:
        if use_engine and a.engine_eval_cp is not None:
            cp = float(a.engine_eval_cp)  # white-perspective
            return cp if pov == chess.WHITE else -cp
        return float(a.score_after)

    def _best_value(self, analyses: List[CandidateAnalysis], use_engine: bool, pov: bool) -> float:
        vals = [self._value_for_ranking(a, use_engine=use_engine, pov=pov) for a in analyses]
        return max(vals)

    def _comment_ja(self, a: CandidateAnalysis, risk_cat: str) -> str:
        d = a.deltas.copy()
        items = sorted(d.items(), key=lambda kv: (-abs(kv[1]), kv[0]))[:3]

        def fmt_feat(name: str) -> str:
            return {
                "material": "駒得",
                "king_safety": "玉の安全",
                "activity": "活動性",
                "center_control": "中央支配",
                "pawn_structure": "歩構造",
                "piece_safety": "駒の安全度",
                "threats": "脅威",
            }.get(name, name)

        if risk_cat == "low_risk_stable":
            head = "ほぼ最善級の安定手。"
        elif risk_cat == "balanced":
            head = "実戦的だがトレードオフが出る手。"
        else:
            head = "見劣りしやすい勝負手寄り（リスク高）。"

        reasons = []
        for k, v in items:
            if abs(v) < 0.08:
                continue
            arrow = "↑" if v > 0 else "↓"
            reasons.append(f"{fmt_feat(k)}{arrow}{abs(v):.2f}")

        core = " / ".join(reasons) if reasons else "差分小（性質が近い）"
        evr = a.evr
        evr_txt = f"EVR≈[{evr.low:.2f}, {evr.high:.2f}]（中心{evr.base:.2f}）"
        return f"{head} 主因: {core}。{evr_txt}"

    def _build_hds_log(
        self,
        board0: chess.Board,
        perspective: ColorStr,
        use_engine: bool,
        score_before: float,
        weights: ScoreWeights,
        analyses: List[CandidateAnalysis],
        max_alternatives: int,
    ) -> str:
        pov = chess.WHITE if perspective.lower() == "white" else chess.BLACK

        lines: List[str] = []
        lines.append("=== HDS DECISION LAYER LOG (CHESS) ===")
        lines.append(f"Perspective: {perspective.upper()} | Turn: {'WHITE' if board0.turn == chess.WHITE else 'BLACK'}")
        lines.append("-" * 56)

        lines.append("[LAYER 1: OBJECTIVE]")
        lines.append(">> 主目的: 期待スコア（勝ちやすさ）の最大化（説明可能な特徴量スコア）。")
        if use_engine:
            lines.append(">> 補助情報: エンジン評価（cp）を順位付けの一次根拠として使用（ログ接続モード）。")
        else:
            lines.append(">> 補助情報: エンジン不使用。全合法手を軽量特徴量で評価し上位候補を生成。")
        lines.append(f">> 現在スコア（before）: {float(round(score_before, 4)):.4f}")
        lines.append("")

        lines.append("[LAYER 2: EVALUATION AXES]")
        w = weights.as_dict()
        for k in ["material", "king_safety", "activity", "center_control", "pawn_structure", "piece_safety", "threats"]:
            lines.append(f"* {k}: weight={w[k]:.2f}")
        lines.append("")

        lines.append("[LAYER 3: ALTERNATIVES ANALYSIS]")
        take = analyses[: max_alternatives]
        for a in take:
            rank_val = self._value_for_ranking(a, use_engine=use_engine, pov=pov)
            ev_note = f"engine_cp={a.engine_eval_cp}" if a.engine_eval_cp is not None else "engine_cp=N/A"
            lines.append(
                f">> {a.move_san} ({a.move_uci}) | rank_value={rank_val:.2f} | "
                f"score_after={a.score_after:.2f} | delta={a.score_delta:+.2f} | {ev_note}"
            )
            items = sorted(a.deltas.items(), key=lambda kv: (-abs(kv[1]), kv[0]))[:3]
            dtxt = ", ".join([f"{k}:{v:+.2f}" for k, v in items])
            lines.append(f"   deltas: {dtxt}")
            lines.append(f"   comment: {a.comment_ja}")
        lines.append("")

        lines.append("[LAYER 4: RISK (EVR) ASSESSMENT]")
        for a in take:
            lines.append(
                f">> {a.move_san} ({a.move_uci}): gap_to_best={a.gap_to_best:.1f}cp "
                f"| risk={a.risk_category.upper()} | EVR=[{a.evr.low:.2f}, {a.evr.high:.2f}]"
            )
        lines.append("")

        best = analyses[0]
        lines.append("[LAYER 5: FINAL CHOICE]")
        lines.append(f">> 選択手: {best.move_san} ({best.move_uci})")
        lines.append(f">> 理由: rank_value最大、かつリスクカテゴリ={best.risk_category}。")
        lines.append(">> 補足: 本デモは“構造化＋ログ翻訳”まで（内部重み推定の提示はしない）。")

        return "\n".join(lines)

    def _build_executive_summary_ja(
        self,
        perspective: ColorStr,
        use_engine: bool,
        analyses: List[CandidateAnalysis],
    ) -> str:
        best = analyses[0]
        others = analyses[1:]

        parts: List[str] = []
        parts.append(f"視点={perspective}。推奨は **{best.move_san}（{best.move_uci}）**。")
        parts.append(
            f"{best.risk_category} / Δ={best.score_delta:+.2f} / EVR≈[{best.evr.low:.2f}, {best.evr.high:.2f}]"
        )
        if use_engine and best.engine_eval_cp is not None:
            parts.append(f"エンジン評価（参考）: {best.engine_eval_cp}cp（白視点）。")

        if others:
            parts.append("他候補（上位）:")
            for a in others[:2]:
                extra = f", engine={a.engine_eval_cp}cp" if (use_engine and a.engine_eval_cp is not None) else ""
                parts.append(
                    f"- {a.move_san}（{a.move_uci}）: gap_to_best={a.gap_to_best:.1f}cp, {a.risk_category}, Δ={a.score_delta:+.2f}{extra}"
                )

        return "\n".join(parts)


# -----------------------------
# 外部ログの受け口（任意）
# -----------------------------

def candidates_from_engine_log(engine_log: List[Dict[str, Any]]) -> List[CandidateMove]:
    """
    例:
      engine_log = [
        {"move":"e2e4","eval":35,"pv":"e2e4 e7e5 ...","policy":0.31},
        ...
      ]
    eval は cp（白視点 +）想定
    """
    out: List[CandidateMove] = []
    for row in engine_log:
        mv = str(row["move"])
        ev = row.get("eval", None)
        out.append(
            CandidateMove(
                move_uci=mv,
                engine_eval_cp=int(ev) if ev is not None else None,
                policy=float(row["policy"]) if row.get("policy") is not None else None,
                pv_uci=str(row["pv"]) if row.get("pv") is not None else None,
                label=str(row["label"]) if row.get("label") is not None else None,
            )
        )
    return out


# ============================================================
# DEMO
# ============================================================

explainer = HDSChessExplainer()

# Demo 1: 初期局面（エンジン無しで候補自動生成）
fen0 = chess.STARTING_FEN
rep0 = explainer.explain_position(fen0, perspective="white", candidates=None, max_alternatives=5)

print(rep0.hds_log)
print("\n=== EXECUTIVE SUMMARY (JA) ===")
print(rep0.executive_summary_ja)
print("\n=== TOP TABLE (brief) ===")
for a in rep0.table[:5]:
    print(f"{a.move_san:6s} {a.move_uci:6s} gap={a.gap_to_best:6.1f}cp risk={a.risk_category:18s} | {a.comment_ja}")

# Demo 2: “エンジン候補手ログ接続”の例（ダミー）
# engine_candidates = candidates_from_engine_log([
#     {"move": "e2e4", "eval": 35},
#     {"move": "d2d4", "eval": 28},
#     {"move": "g1f3", "eval": 22},
# ])
# rep1 = explainer.explain_position(fen0, perspective="white", candidates=engine_candidates, max_alternatives=3)
# print("\n" + rep1.hds_log)

# Demo 3: 決定性チェック（3回同じ入力→同じ出力）
logs = []
sums = []
for _ in range(3):
    r = explainer.explain_position(fen0, perspective="white", candidates=None, max_alternatives=5)
    logs.append(r.hds_log)
    sums.append(r.executive_summary_ja)
assert logs[0] == logs[1] == logs[2], "HDS Logs are not deterministic!"
assert sums[0] == sums[1] == sums[2], "Summaries are not deterministic!"
print("\nOK: 3回とも完全一致（決定的動作）。")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 67.7 MB/s eta 0:00:00
=== HDS DECISION LAYER LOG (CHESS) ===
Perspective: WHITE | Turn: WHITE
--------------------------------------------------------
[LAYER 1: OBJECTIVE]
>> 主目的: 期待スコア（勝ちやすさ）の最大化（説明可能な特徴量スコア）。
>> 補助情報: エンジン不使用。全合法手を軽量特徴量で評価し上位候補を生成。
>> 現在スコア（before）: 2.8935

[LAYER 2: EVALUATION AXES]
* material: weight=1.00
* king_safety: weight=0.85
* activity: weight=0.45
* center_control: weight=0.25
* pawn_structure: weight=0.25
* piece_safety: weight=0.55
* threats: weight=0.35

[LAYER 3: ALTERNATIVES ANALYSIS]
>> Nc3 (b1c3) | rank_value=3.47 | score_after=3.47 | delta=+0.57 | engine_cp=N/A
   deltas: center_control:+2.00, activity:+0.16, king_safety:+0.00
   comment: ほぼ最善級の安定手。 主因: 中央支配↑2.00 / 活動性↑0.16。EVR≈[3.47, 3.81]（中心3.47）
>> Nf3 (g1f3) | rank_value=3.47 | score_after=3.47 | delta=+0.57 | engine_cp=N/A
   deltas: center_control:+2.00, activity:+0.16, king_safety:+0.00
   comment: ほぼ最善級の安定手。 主因: 中央支配↑2.00 / 活動性↑0.16。EV